In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [2]:
val spark = SparkSession.builder()
      .appName("CaseStudy4 - Duplicate Record Removal Pipeline")
      .getOrCreate()

spark = org.apache.spark.sql.SparkSession@68cd7b09


org.apache.spark.sql.SparkSession@68cd7b09

In [3]:
val moviesDataPath = "gs://spark-tasks-bucket/day_16_17/movie.csv"
val moviesDF = spark.read.option("header", "true").option("inferSchema", "true").csv(moviesDataPath)

moviesDataPath = gs://spark-tasks-bucket/day_16_17/movie.csv
moviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [4]:
moviesDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [5]:
val count = moviesDF.count()

27278

count = 27278


In [6]:
val clearedRDD = moviesDF.dropDuplicates("movieId", "title").count()

clearedRDD = 27278


27278

Syntax Error.: 

In [8]:
val moviesIn1971 = moviesDF.filter(col("title").contains("(1971)"))
val moviesIn1971Count = moviesIn1971.count()

moviesIn1971 = [movieId: int, title: string ... 1 more field]
moviesIn1971Count = 205


205

In [9]:
val unionDF = moviesDF.union(moviesIn1971)
val unionDFCount = unionDF.count()

unionDF = [movieId: int, title: string ... 1 more field]
unionDFCount = 27483


27483

In [10]:
// Now save this file to hdfs
val hdfsPath = "hdfs:///user/day_16_17/duplicate_movies"

hdfsPath = hdfs:///user/day_16_17/duplicate_movies


hdfs:///user/day_16_17/duplicate_movies

In [11]:
unionDF.write.mode("overwrite").csv(hdfsPath)

In [12]:
val moviesSchema = StructType(Array(
  StructField("movieId", IntegerType, nullable = true),
  StructField("title", StringType, nullable = true),
  StructField("genres", StringType, nullable = true)
))

val moviesUpdatedDF = spark.read.schema(moviesSchema).option("header", "false").csv(hdfsPath)

moviesSchema = StructType(StructField(movieId,IntegerType,true),StructField(title,StringType,true),StructField(genres,StringType,true))
moviesUpdatedDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [13]:
moviesUpdatedDF.count()

27483

In [14]:
moviesDF.dropDuplicates("movieId", "title").count()

27278

In [15]:
27483 - 27278
// This means there are 205 duplicate records

205

In [16]:
val rdd = moviesUpdatedDF.rdd.map {row =>
    val movieId = row.getAs[Int]("movieId")
    val title = row.getAs[String]("title")
    
    ((movieId, title), row)
}.reduceByKey((r1, _) => r1)
.map{ case (_, row) => row }

rdd = MapPartitionsRDD[75] at map at <console>:40


MapPartitionsRDD[75] at map at <console>:40

In [17]:
rdd.count()

27278

In [18]:
// The rdd created from moviesUpdatedDF(records: 27483) has now filtered to rdd(records: 27278) removing duplicate 
// based on (movieId and title)

Syntax Error.: 

In [19]:
val gsPath = "gs://spark-tasks-bucket/day_16_17/cleaned_movies"

gsPath = gs://spark-tasks-bucket/day_16_17/cleaned_movies


gs://spark-tasks-bucket/day_16_17/cleaned_movies

In [20]:
rdd.map {row =>
    val movieId = row.getAs[Int]("movieId")
    val title = row.getAs[String]("title")
    val genres = row.getAs[String]("genres")
    
    (movieId, title, genres)
}.toDF("movieId", "title", "genres")
.write
.format("avro")
.mode("overwrite")
.save(gsPath)

In [22]:
// Read from this path to ensure data is saved as expected
val validationDF = spark.read.format("avro").load(gsPath)

validationDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [24]:
validationDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|  82108|Against the Curre...|               Drama|
|  94133|  Hammer, The (2010)|               Drama|
|  91284|Lonely Passion of...|       Drama|Romance|
|  96717|Pearls of the Cro...|              Comedy|
|  69042|Flash Gordon's Tr...|       Action|Sci-Fi|
|   3530|Smoking/No Smokin...|              Comedy|
|  73449| V.I.P.s, The (1963)|               Drama|
|  27783|Lost Embrace (Abr...|        Comedy|Drama|
|  32369|Panic in the Stre...|Crime|Drama|Film-...|
|   4077|With a Friend Lik...|      Drama|Thriller|
| 127248|  The Auction (2013)|               Drama|
|   1584|      Contact (1997)|        Drama|Sci-Fi|
|  89896|Turin Horse, The ...|               Drama|
|  73392|     Collapse (2009)|         Documentary|
|  85378|Mother Carey's Ch...|       Drama|Romance|
|   5131|How to Kill Your ...|        Comedy|Drama|
|  75421|Gir

In [25]:
validationDF.count()

27278

In [ ]:
spark.stop()